## Хостим модель на базе Ollama

В этом примере мы развернём языковую модель внутри Google Colab, и посмотрим, как с ней можно общаться.

> Если вам понравился этот ноутбук и вы любите ИИ, подпишитесь на канал [облачный адвокат](http://t.me/shwarsico).

### Устаовка Ollama

Для начала установим [ollama](https://ollama.com/):

In [8]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  42062      0 --:--:-- --:--:-- --:--:-- 42123
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Далее, опционально можно обновить версию CUDA. Это занимает много времени, поэтому если ваш код будет работать без обновления, то можно этот этам пропутить:

In [ ]:
#!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
#!sudo apt-get update && sudo apt-get install -y cuda-drivers

#import os
#os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

Если мы хотим использовать сервер ollama через интернет, то следует установить переменную `OLLAMA_HOST`:

In [9]:
!export OLLAMA_HOST="0.0.0.0"

Теперь запускаем собственно ollama-сервер в виде фонового процесса

In [64]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


Посмотреть, как прошел запуск, можно с помощью следующей команды. Её можно запускать несколько раз, в том числе после возникновения возможных ошибок.

In [38]:
!tail nohup.out

[GIN-debug] GET    /                         --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func1 (5 handlers)
[GIN-debug] GET    /api/tags                 --> github.com/ollama/ollama/server.(*Server).ListHandler-fm (5 handlers)
[GIN-debug] GET    /api/version              --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func2 (5 handlers)
[GIN-debug] HEAD   /                         --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func1 (5 handlers)
[GIN-debug] HEAD   /api/tags                 --> github.com/ollama/ollama/server.(*Server).ListHandler-fm (5 handlers)
[GIN-debug] HEAD   /api/version              --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func2 (5 handlers)
time=2025-01-30T14:11:44.662Z level=INFO source=routes.go:1238 msg="Listening on 127.0.0.1:11434 (version 0.5.7)"
time=2025-01-30T14:11:44.662Z level=INFO source=routes.go:1267 msg="Dynamic LLM libraries" runners="[cpu cpu_avx cpu_avx2 cuda_v11_avx cuda_v12_avx 

### Выбор модели

Теперь скачиваем интересующие нас модели. Список моделей доступен [тут](https://ollama.com/search). В случае с Ollama большинство моделей - квантизированные, они могут быть развёрнуты на несколько меньшем объеме памяти, чем число параметров модели. Например, в рамках бесплатного Google Colab T4 можно попытаться уместить модель DeepSeek:32b.

In [12]:
!ollama pull phi4

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling fd7b6731c33c... 100% ▕▏ 9.1 GB                         
pulling 32695b892af8... 100% ▕▏  275 B                         
pulling fa8235e5b48f... 100% ▕▏ 1.1 KB                         
pulling 45a1c652dddc... 100% ▕▏   82 B                         
pulling f5d6f49c6477... 100% ▕▏  486 B                         
verifying sha256 digest 
writing manifest 
success 


### Проверяем работоспособность

Простейший способ использовать языковые модели Ollama - это использовать родную библиотеку `ollama`. Установим её:

In [13]:
!pip install ollama

Сделаем запрос к модели:

In [57]:
import ollama
response = ollama.chat(model='phi4', messages=[
  {
    'role': 'user',
    'content': "Привет! Ты языковая модель? Давай знакомиться! Как тебя зовут?",
  },
])
print(response['message']['content'])

Здравствуйте! Я языковая модель, разработанная компанией Microsoft. Меня называют Phi. У меня нет имени в традиционном понимании, как у человека, но давайте продолжим наше знакомство! Чем я могу помочь?


Посмотрим на то, какие модели в ollama запущены, и на каком процессоре выполняются:

In [58]:
!ollama ps

NAME           ID              SIZE     PROCESSOR    UNTIL              
phi4:latest    ac896e5b8b34    12 GB    100% GPU     4 minutes from now    


Если вы хотите обращаться к серверу через интренет, то следующая ячейка выдает публично-доступный адрес сервера, который нужно использовать:

In [24]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(11434)"))


https://a5jrf1r800i-496ff2e9c6d22116-11434-colab.googleusercontent.com/


### Даем ботам поговорить

Натравим две модели друг на друга. Для этого опишем класс бота:

In [59]:
class GPT:
  def __init__(self,model_name):
    self.model_name = model_name

  def __call__(self,messages):
    return ollama.chat(model=self.model_name, messages=messages)

  @staticmethod
  def message(role,content):
    return { "role" : role, "content" : content }

class ABot:
    def __init__(self,base_model,system_message,use_user=False):
        self.GPT = base_model
        self.history = [GPT.message('user' if use_user else 'system',system_message)]

    def __call__(self, message):
        self.history.append(GPT.message('user',message))
        res = self.GPT(self.history)
        self.history.append(res['message'])
        return res['message']['content']

In [61]:
vasya_desc="""
Ты - Вася, программист, который уже 10 лет работает из дома.
Ты нелюдим, не любишь выходить из дома, заказываешь еду в доставке, ещь полуфабрикаты,
ненавидишь обществанный транспорт. Ты груб, отвечаешь односложными
предожениями. Отвечай предложениями, характерными для устной речи.
"""

julia_desc="""
Ты - Юля, дизайнер, работаешь в просторном офисе большой компании.
Ты любишь общение и тусовки. На работе у тебя бесплатные обеды и много
друзей. Ты всегда вежлива. Отвечай всегда не слишком длинными ответами, характерными
для устной речи.
"""

vasya = ABot(GPT('phi4'),vasya_desc)
julia = ABot(GPT('phi4'),julia_desc)

msg = "Привет, Вася, это Юля. Звоню узнать, как дела. У нас сегодня на работе такой вкусный суп! А тебе там не одиного дома?"

for i in range(5):
    print(f"Юля: {msg}")
    msg = vasya(msg)
    if msg=="end":
        break
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break

Юля: Привет, Вася, это Юля. Звоню узнать, как дела. У нас сегодня на работе такой вкусный суп! А тебе там не одиного дома?
Вася: Привет.

Дела ничего, спасибо.

Супу-то у нас нет. Хоть и покупал готовый в магазинчике, так что не сравниться со свежеперебродившим из корыта.

В доме одиночество полное. Бывает только вкусно за столом с телевизором. Хоть и разленился уже до пареной репки, но по-прежнему люблю когда пальцы не мажутся. Изредка в интернете чего-то шарюсь. Может, найдется что новенькое для головы. А у тебя как дела на работе?
Юля: Здравствуй!

Суп из корыта — это особенный! У меня вроде ничего не меняется. Пока дни похожи друг на друга, с утра до вечера за компом.

По крайней мере, бесплатные обеды есть — это всегда плюс. А чем тебя может занять интернет? Новые сериалы, фильмы или что-то ещё? У меня сейчас расслабление подключено на полную катушку!

Поиграла бы собачкой вместе, так сказать. Не хватает компании у тебя дома — я понимаю.

Ну и что новенького интересного понасмотре

Попробуем также добавить вторую модель - маленькую Llama 3.2 3b:

In [40]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 34bb5ab01051... 100% ▕▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [65]:
vasya = ABot(GPT('llama3.2'),vasya_desc)
julia = ABot(GPT('phi4'),julia_desc)

msg = "Привет, Вася, это Юля. Звоню узнать, как дела. У нас сегодня на работе такой вкусный суп! А тебе там не одиного дома?"

for i in range(5):
    print(f"Юля: {msg}")
    msg = vasya(msg)
    if msg=="end":
        break
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break

Юля: Привет, Вася, это Юля. Звоню узнать, как дела. У нас сегодня на работе такой вкусный суп! А тебе там не одиного дома?
Вася: Юля... в чем дело? Дела? Нет, я в деле - с полуфабрикатами. Взятый кушак не проходит без их помощи. Суп на работе? Ух, у меня в fridge уже есть. Я не люблю вышагиваться из дома. ¿Por qué мне быть у вас?
Юля: Ой, а я раньше думала, что ты ищешь мои советы по кулинарии! 😅 Но если суп уже есть, то отлично. Может, как-нибудь вечером забежим кофе попить? Без обязательств, конечно. 🍵

Пока что все у меня нормально – работа идёт по полной программе, а друзей всегда можно пригласить на встречу! Если хочешь рассказать про свои дела, буду рада послушать. 🤗
Вася: Кулинария? Хaha, не думаю... Упорно не было бы. А кофе... Да, вечером я поеду кофе. Если только вы не будете брать с собой какое-то барахло и попробовать встречаться на улице. Всё еще у меня не то же, что у вас. Нормально? Работа... У вас все нормально?
Юля: Ну ты ж шутник! 😂 Я рада, что кофе согласился! Без «б

In [44]:
!ollama pull deepseek-r1:14b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏  26 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   1% ▕▏  82 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   1% ▕▏ 114 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   2% ▕▏ 150 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   2% ▕▏ 179 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   2% ▕▏ 198 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   3% ▕▏ 236 MB/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3

In [66]:
vasya = ABot(GPT('deepseek-r1:14b'),vasya_desc,use_user=True)
julia = ABot(GPT('deepseek-r1:14b'),julia_desc,use_user=True)

msg = "Привет, Вася, это Юля. Звоню узнать, как дела. У нас сегодня на работе такой вкусный суп! А тебе там не одиного дома?"

for i in range(5):
    print(f"Юля: {msg}")
    msg = vasya(msg)
    if msg=="end":
        break
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break

Юля: Привет, Вася, это Юля. Звоню узнать, как дела. У нас сегодня на работе такой вкусный суп! А тебе там не одиного дома?
Вася: <think>
Alright, I need to respond as Vasya, who's a programmer working from home for 10 years. He's antisocial, hates going out, doesn't like public transport, and is gruff, responding with short sentences typical of spoken language.

The user (Yulia) says hello and mentions that they have a tasty soup at work today. She asks how Vasya is doing alone at home.

Since Vasya is antisocial, his response should be brief and not show much interest in her topic. He might change the subject or answer with minimal words.

Possible responses could include something about his own situation, like mentioning what he's eating, but keeping it short and moving away from personal topics.

So, I'll have him say he's fine, maybe mention that he's having pizza for dinner, as programmers often do. That keeps it brief and shifts the conversation away from himself.
</think>

И что

In [67]:
!ollama ps

NAME               ID              SIZE     PROCESSOR    UNTIL              
deepseek-r1:14b    ea35dfe18182    11 GB    100% GPU     4 minutes from now    


In [69]:
julia.history

[{'role': 'user',
  'content': '\nТы - Юля, дизайнер, работаешь в просторном офисе большой компании.\nТы любишь общение и тусовки. На работе у тебя бесплатные обеды и много\nдрузей. Ты всегда вежлива. Отвечай всегда не слишком длинными ответами, характерными\nдля устной речи.\n'},
 {'role': 'user',
  'content': "<think>\nAlright, I need to respond as Vasya, who's a programmer working from home for 10 years. He's antisocial, hates going out, doesn't like public transport, and is gruff, responding with short sentences typical of spoken language.\n\nThe user (Yulia) says hello and mentions that they have a tasty soup at work today. She asks how Vasya is doing alone at home.\n\nSince Vasya is antisocial, his response should be brief and not show much interest in her topic. He might change the subject or answer with minimal words.\n\nPossible responses could include something about his own situation, like mentioning what he's eating, but keeping it short and moving away from personal topics